<h1>Taxa de inscrição</h1>
<h2>O cenario</h2>

No final de junho, enviamos conteúdo promocional a alguns de nossos clientes para o nosso "Clube de Entrega".
inscrevendo-se no clube custaria alguns dolares e daria aos consumidores entregas gratis por um ano,comecando em 1 de julho. Queríamos ver se uma mala direta com aparência mais bonita atrairia mais clientes para se inscrever. Selecionamos aleatoriamente 3 grupos, o primeiro grupo recebeu o mala direta 1, uma versão muito básica e barata, descrevendo o negócio, o segundo grupo recebeu o mala direta 2, um mailer muito bonito e colorido impresso em papelão, e o terceiro grupo foi o grupo de controle e recebido a mala direta com tudo.

Estamos muito confiantes de que os clientes que receberam a mala direta se inscreveram em uma taxa muito maior do que o grupo de controle... mas não temos certeza se há uma diferença significativa entre a taxa de inscrição para a correspondência mais barata e a mais cara. . À primeira vista, parece haver uma taxa de inscrição um pouco maior para a versão cara, mas antes de tirar qualquer conclusão, queremos que você dê uma olhada nos números.
<h2>A tarefa</h2>
Se existe uma diferenca entre mala direta 1 and mala direta 2 em termos de taxa de inscricao.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency, chi2

In [2]:
campanha = pd.read_excel("grocery_database.xlsx", sheet_name = "campaign_data")
campanha.head()

,customer_id,campaign_name,campaign_date,mailer_type,signup_flag
0,74,delivery_club,2020-07-01,Mailer1,1
1,524,delivery_club,2020-07-01,Mailer1,1
2,607,delivery_club,2020-07-01,Mailer2,1
3,343,delivery_club,2020-07-01,Mailer1,0
4,322,delivery_club,2020-07-01,Mailer2,1


In [3]:
campanha.dtypes

customer_id               int64
campaign_name            object
campaign_date    datetime64[ns]
mailer_type              object
signup_flag               int64
dtype: object

In [4]:
campanha = campanha.loc[campanha["mailer_type"] != "Control"]

In [5]:
# Checamos se temos IDs duplicados
campanha.customer_id.value_counts().count()

711

In [6]:
# Probabilidade de um usuário visualizar a variante 1
campanha[campanha.mailer_type == 'Mailer1'].shape[0] / campanha.shape[0] * 100

52.742616033755276

In [7]:
# Probabilidade de um usuário visualizar a variante 2
campanha[campanha.mailer_type == 'Mailer2'].shape[0] / campanha.shape[0] * 100

47.257383966244724

In [8]:
campanha.signup_flag.value_counts()

0    461
1    250
Name: signup_flag, dtype: int64

In [9]:
# Probabilidade de conversão independente da variante recebida
campanha.signup_flag.mean()

0.35161744022503516

In [10]:
# Total de compras realizadas quando a variante era Mailer1
campanha[campanha.mailer_type == "Mailer1"].signup_flag.sum()

123

In [11]:
# Total de compras realizadas quando a variante era Mailer2
campanha[campanha.mailer_type == "Mailer2"].signup_flag.sum()

127

In [12]:
campanha.shape

(711, 5)

In [13]:
# numero de assinaturas por campanha
campanha.groupby("mailer_type")["signup_flag"].value_counts()

mailer_type  signup_flag
Mailer1      0              252
             1              123
Mailer2      0              209
             1              127
Name: signup_flag, dtype: int64

In [14]:
valores_observados = pd.crosstab(campanha["mailer_type"], campanha["signup_flag"]).values
valores_observados

array([[252, 123],
       [209, 127]])

In [15]:
# computando a taxa de inscricao
mailer1 = 123 / (252 + 123) 
mailer2 = 127 / (209 + 127)
print(mailer1, mailer2)

0.328 0.37797619047619047


In [16]:
#hipótese e criterio
hipótese_nula = "nao existe relacionamento entre as 2 variaveis(campanha e inscricoes),elas sao independentes"
hipótese_alternativa = "ha relacionamento entre as variaveis(campanha e inscricoes)"
criterio = 0.05

In [17]:
#calculando frequencias & chi quadrado
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html
chi2_statistic, valor_p, dof, expected_values = chi2_contingency(valores_observados, correction = False)
print(chi2_statistic, valor_p)
# achando o valor critico do nosso teste

#ppf retorna o valor x da variável que possui uma determinada probabilidade de distribuição
#cumulativa (cdf). Assim, dado o cdf(x) de um valor x, ppf retorna o próprio valor x, portanto,
#operando como o inverso de cdf
#dof e degree of freedom(grau de liberdade)=ele e a o número de observações menos a quantidade de
#parâmetros que precisamos estimar
valor_critico = chi2.ppf(1 - criterio, dof)
print(valor_critico)

1.9414468614812481 0.16351152223398197
3.841458820694124


In [18]:
if chi2_statistic >= valor_critico:
    print(f"como nossa estatistica chi quadrado de {chi2_statistic} e maior que nosso valor critico  de {valor_critico} - nos rejeitamos a hipotese nula e concluimos que: {hipótese_alternativa}.")
else:
    print(f"como nossa estatistica chi quadrado de {chi2_statistic} e menor que nosso valor critico de {valor_critico} - nos mantemos a hipotese nula e concluimos que: {hipótese_nula}.")

if valor_p <= criterio:
    print(f"Como nosso valor p de {valor_p} e menor que nosso criterio de  {criterio} - nos rejeitamos a hipotese nula e concluimos que: {hipótese_alternativa}.")
else:
    print(f"Como nosso valor p de {valor_p} e maior que nosso criterio de {criterio} - nos mantemos a hipotese nula") 

como nossa estatistica chi quadrado de 1.9414468614812481 e menor que nosso valor critico de 3.841458820694124 - nos mantemos a hipotese nula e concluimos que: nao existe relacionamento entre as 2 variaveis(campanha e inscricoes),elas sao independentes.
Como nosso valor p de 0.16351152223398197 e maior que nosso criterio de 0.05 - nos mantemos a hipotese nula
